In [2]:
import sys

sys.path.insert(0, "../..")
from pathlib import Path
from tentamen.settings import GRUAttationSearchSpace


In [3]:
tune_dir = Path("../../logsGRUAtt/train_2023-01-28_20-11-02")
tune_dir.exists()


True

In [4]:
from ray.tune import ExperimentAnalysis
import ray
ray.init(ignore_reinit_error=True)

analysis = ExperimentAnalysis(tune_dir)


2023-01-29 16:09:00,561	INFO worker.py:1538 -- Started a local Ray instance.
2023-01-29 16:09:06,753	INFO experiment_analysis.py:795 -- No `self.trials`. Drawing logdirs from checkpoint file. This may result in some information that is out of sync, as checkpointing is periodic.


In [139]:
analysis.results_df.columns

Index(['iterations', 'train_loss', 'test_loss', 'Accuracy', 'time_this_iter_s',
       'done', 'timesteps_total', 'episodes_total', 'training_iteration',
       'experiment_id', 'date', 'timestamp', 'time_total_s', 'pid', 'hostname',
       'node_ip', 'time_since_restore', 'timesteps_since_restore',
       'iterations_since_restore', 'warmup_time', 'experiment_tag',
       'config/input', 'config/output', 'config/tunedir', 'config/hidden_size',
       'config/num_layers', 'config/num_heads', 'config/dropout'],
      dtype='object')

In [6]:
import plotly.express as px

plot = analysis.results_df
select = ["Accuracy", "config/hidden_size", "config/dropout", "config/num_layers","config/num_heads"]
p = plot[select].reset_index().dropna()


Let's sort by accuracy

In [38]:
p.sort_values("Accuracy", inplace=True)

p

,trial_id,Accuracy,config/hidden_size,config/dropout,config/num_layers,config/num_heads
13,62592893,0.146599,208,0.126589,13,8
32,da0eecc6,0.187500,176,0.030048,14,2
34,daf56522,0.191636,112,0.075183,14,8
35,dafedab5,0.192096,272,0.184523,12,4
33,da545592,0.205882,80,0.152158,14,4
3,1953ea3c,0.227022,192,0.064596,12,8
15,8250d7cf,0.253676,144,0.133185,11,8
5,1d7be642,0.280790,240,0.094064,15,4
7,20e5113b,0.424632,192,0.013996,10,2
1,11182a64,0.540441,96,0.015573,8,8


In [136]:
px.parallel_coordinates(p, color="Accuracy")


In [135]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

columns = p.iloc[:,2:].columns.to_list()

set_rows = 2
set_cols = 2
row_count = 0
col_count = 1

fig = make_subplots(
    rows=set_rows,
     cols=set_cols,
     subplot_titles=([name.split("/",1)[1] for name in columns])
)

for num,column in enumerate(columns):
    col_name = column.split("/",1)[1]
    if (num)% 2 == 0:
        row_count += 1
        col_count = 1
    else:
        col_count += 1

    fig.add_trace(
        go.Scatter(
                x=p[column],
                y=p["Accuracy"],
                mode="markers"
        ),
        row=row_count,
        col=col_count
    )

    fig.update_xaxes(title_text=col_name, row=row_count, col=col_count)
    fig.update_yaxes(title_text="Accuracy", row=row_count, col=col_count)

fig.update_layout(
    height=600,
    width=800,
    title_text="Parameters vs accuarcy",
    showlegend=False
)

fig.show()


In [32]:
# Best trial:
analysis.get_best_trial(metric="test_loss", mode="min")


train_b5273c64

In [137]:
p[-1:]


,trial_id,Accuracy,config/hidden_size,config/dropout,config/num_layers,config/num_heads
25,b5273c64,0.987592,160,0.020913,2,2


Or the best config

In [27]:
analysis.get_best_config(metric="Accuracy", mode="max")


{'input': 13,
 'output': 20,
 'tunedir': WindowsPath('C:/Users/gvanh/Git_documents/ML22-tentamen/logsGRUAtt'),
 'hidden_size': 160,
 'num_layers': 2,
 'num_heads': 2,
 'dropout': 0.020912795763146617}